In [3]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50

try:
    scraper = sfc.FBRef()
    df = scraper.scrape_gk(2019, "EPL", normalize=True, player=True)
    scraper.close()
except:
    traceback.print_exc()
    scraper.close()

In [4]:
df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Rk             Player             Nation   
0                 1.0            Alisson             br BRA   
1                 2.0  Kepa Arrizabalaga             es ESP   
2                 3.0      Asmir Begović             ba BIH   
3                 4.0  Marcus Bettinelli            eng ENG   
4                 5.0        Artur Boruc             pl POL   
5                 6.0       David Button            eng ENG   
6                 7.0    Willy Caballero             ar ARG   
7                 8.0          Petr Čech             cz CZE   
8                 9.0       Joel Coleman            eng ENG   
9                10.0    Martin Dúbravka             sk SVK   
10               11.0            Ederson             br BRA   
11               12.0     Neil Etheridge             ph PHI   
12               13.0   Łukasz Fabiański             pl POL   
13               14.0           Fabricio             es ESP   
14               15.0     Fraser Forster            eng ENG   
15               16.0         Ben Foster            eng ENG   
16               17.0    Paulo Gazzaniga             ar ARG   
17               18.0       David de Gea             es ESP   
18               19.0     Vicente Guaita             es ESP   
19               20.0         Angus Gunn            eng ENG   
20               21.0          Ben Hamer            eng ENG   
21               22.0           Joe Hart            eng ENG   
22               23.0         Tom Heaton            eng ENG   
23               24.0    Wayne Hennessey            wal WAL   
24               25.0         Bernd Leno             de GER   
25               26.0        Hugo Lloris             fr FRA   
26               27.0        Jonas Lössl             dk DEN   
27               28.0      Alex McCarthy            eng ENG   
28               29.0        Will Norris            eng ENG   
29               30.0       Rui Patrício             pt POR   
30               31.0    Jordan Pickford            eng ENG   
31               32.0        Sergio Rico             es ESP   
32               33.0         John Ruddy            eng ENG   
33               34.0        Mathew Ryan             au AUS   
34               35.0  Kasper Schmeichel             dk DEN   
35               36.0     Julián Speroni             ar ARG   
36               37.0       Mark Travers             ie IRL   
37               38.0        Michel Vorm             nl NED   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                  Pos              Squad                Age   
0                  GK          Liverpool               25.0   
1                  GK            Chelsea               23.0   
2                  GK        Bournemouth               31.0   
3                  GK             Fulham               26.0   
4                  GK        Bournemouth               38.0   
5                  GK           Brighton               29.0   
6                  GK            Chelsea               36.0   
7                  GK            Arsenal               36.0   
8                  GK       Huddersfield               22.0   
9                  GK      Newcastle Utd               29.0   
10                 GK    Manchester City               24.0   
11                 GK       Cardiff City               28.0   
12                 GK           West Ham               33.0   
13                 GK             Fulham               30.0   
14                 GK        Southampton               30.0   
15                 GK            Watford               35.0   
16                 GK          Tottenham               26.0   
17                 GK     Manchester Utd               27.0   
18                 GK     Crystal Palace               31.0   
19                 GK        Southampton               22.0   
20                 GK       Huddersfield               30.0   
21                 GK            